In [24]:
# import socket
# fimport socks
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import date
import time
from pandas import DataFrame
import os
import pandas as pd

In [2]:
# def connectTor():
#   ## Connect to Tor for privacy purposes
#      socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, '127.0.0.1', 9150, True)
#      socket.socket = socks.socksocket
#      print "connected to Tor!"

def soup_url(type_of_line, tdate = str(date.today()).replace('-','')):
## get html code for odds based on desired line type and date
    if type_of_line == 'Spreads':
        url_addon = ''
    elif type_of_line == 'ML':
        url_addon = 'money-line/'
    elif type_of_line == 'Totals':
        url_addon = 'totals/'
    # elif type_of_line == '1H':
        # url_addon = '1st-half/'
    # elif type_of_line == '1HRL':
        # url_addon = 'pointspread/1st-half/'
    # elif type_of_line == '1Htotal':
        # url_addon = 'totals/1st-half/'
    else:
        print("Wrong url_addon")
    url = 'https://classic.sportsbookreview.com/betting-odds/nba-basketball/' + url_addon + '?date=' + tdate
    now = datetime.datetime.now()
    raw_data = requests.get(url)
    soup_big = BeautifulSoup(raw_data.text, 'html.parser')
    soup = soup_big.find_all('div', id='OddsGridModule_5')[0]
    timestamp = time.strftime("%H:%M:%S")
    return soup, timestamp

def parse_and_write_data(soup, date, time, not_ML = True):
## Parse HTML to gather line data by book
    def book_line(book_id, line_id, homeaway):
        ## Get Line info from book ID
        line = soup.find_all('div', attrs = {'class':'el-div eventLine-book', 'rel':book_id})[line_id].find_all('div')[homeaway].get_text().strip()
        return line
    '''
    BookID  BookName
    238     Pinnacle
    19      5Dimes
    93      Bookmaker
    1096    BetOnline
    169     Heritage
    123     BetDSI
    999996  Bovada
    139     Youwager
    999991  SIA
    '''
    if not_ML:
        df = DataFrame(
                columns=('key','date','time',
                         'team','opp_team','pinnacle_line','pinnacle_odds',
                         '5dimes_line','5dimes_odds',
                         'heritage_line','heritage_odds',
                         'bovada_line','bovada_odds',
                         'betonline_line','betonline_odds'))
    else:
        df = DataFrame(
            columns=('key','date','time',
                     'team',
                     'opp_team',
                     'pinnacle','5dimes',
                     'heritage','bovada','betonline'))
    counter = 0
    number_of_games = len(soup.find_all('div', attrs = {'class':'el-div eventLine-rotation'}))
    for i in range(0, number_of_games):
        A = []
        H = []
        print(str(i+1)+'/'+str(number_of_games))
        
        ## Gather all useful data from unique books
        # consensus_data = 	soup.find_all('div', 'el-div eventLine-consensus')[i].get_text()
        info_A = 		        soup.find_all('div', attrs = {'class':'el-div eventLine-team'})[i].find_all('div')[0].get_text().strip()
        # hyphen_A =              info_A.find('-')
        # paren_A =               info_A.find("(")
        team_A =                info_A
        # pitcher_A =             info_A[hyphen_A + 2 : paren_A - 1]
        # hand_A =                info_A[paren_A + 1 : -1]
        ## get line/odds info for unique book. Need error handling to account for blank data
        try:
            pinnacle_A = 	    book_line('238', i, 0)
        except IndexError:
            pinnacle_A = ''
        try:
            fivedimes_A = 	    book_line('19', i, 0)
        except IndexError:
            fivedimes_A = ''
        try:
            heritage_A =        book_line('169', i, 0)
        except IndexError:
            heritage_A = ''
        try:
            bovada_A = 		    book_line('999996', i, 0)
        except IndexError:
            bovada_A = ''
        try:
            betonline_A = 		book_line('1096', i, 0)
        except IndexError:
            betonline_A = ''
        info_H = 		        soup.find_all('div', attrs = {'class':'el-div eventLine-team'})[i].find_all('div')[1].get_text().strip()
        # hyphen_H =              info_H.find('-')
        # paren_H =               info_H.find("(")
        team_H =                info_H
        # pitcher_H =             info_H[hyphen_H + 2 : paren_H - 1]
        # hand_H =                info_H[paren_H + 1 : -1]
        try:
            pinnacle_H = 	    book_line('238', i, 1)
        except IndexError:
            pinnacle_H = ''
        try:
            fivedimes_H = 	    book_line('19', i, 1)
        except IndexError:
            fivedimes_H = ''
        try:
            heritage_H = 	    book_line('169', i, 1)
        except IndexError:
            heritage_H = '.'
        try:
            bovada_H = 		    book_line('999996', i, 1)
        except IndexError:
            bovada_H = '.'
        try:
            betonline_H = 		book_line('1096', i, 1)
        except IndexError:
            betonline_H = ''
        if team_H ==   'Detroit':
            team_H =   'Detroit'
        elif team_H == 'Indiana':
            team_H =   'Indiana'
        elif team_H == 'Brooklyn':
            team_H =   'Brooklyn'
        elif team_H == 'L.A. Lakers':
            team_H =   'L.A. Lakers'
        elif team_H == 'Washington':
            team_H =   'Washington'
        elif team_H == 'Miami':
            team_H =   'Miami'
        elif team_H == 'Minnesota':
            team_H =   'Minnesota'
        elif team_H == 'Chicago':
            team_H =   'Chicago'
        elif team_H == 'Oklahoma City':
            team_H =   'Oklahoma City'
        if team_A ==   'New Orleans':
            team_A =   'New Orleans'
        elif team_A == 'Houston':
            team_A =   'Houston'
        elif team_A == 'Dallas':
            team_A =   'Dallas'
        elif team_A == 'Cleveland':
            team_A =   'Cleveland'
        elif team_A == 'L.A. Clippers':
            team_A =   'L.A. Clippers'
        elif team_A == 'Golden State':
            team_A =   'Golden State'
        elif team_A == 'Denver':
            team_A =   'Denver'
        elif team_A == 'Boston':
            team_A =   'Boston'
        elif team_A == 'Milwaukee':
            team_A =   'Milwaukee'            
       # A.append(str(date) + '_' + team_A.replace(u'\xa0',' ') + '_' + team_H.replace(u'\xa0',' '))
        A.append(date)
        A.append(time)
        A.append('away')
        A.append(team_A)
        # A.append(pitcher_A)
        # A.append(hand_A)
        A.append(team_H)
        # A.append(pitcher_H)
        # A.append(hand_H)
        if not_ML:
            pinnacle_A = pinnacle_A.replace(u'\xa0',' ').replace(u'\xbd','.5')
            pinnacle_A_line = pinnacle_A[:pinnacle_A.find(' ')]
            pinnacle_A_odds = pinnacle_A[pinnacle_A.find(' ') + 1:]
            A.append(pinnacle_A_line)
            A.append(pinnacle_A_odds)
            fivedimes_A = fivedimes_A.replace(u'\xa0',' ').replace(u'\xbd','.5')
            fivedimes_A_line = fivedimes_A[:fivedimes_A.find(' ')]
            fivedimes_A_odds = fivedimes_A[fivedimes_A.find(' ') + 1:]
            A.append(fivedimes_A_line)
            A.append(fivedimes_A_odds)
            heritage_A = heritage_A.replace(u'\xa0',' ').replace(u'\xbd','.5')
            heritage_A_line = heritage_A[:heritage_A.find(' ')]
            heritage_A_odds = heritage_A[heritage_A.find(' ') + 1:]
            A.append(heritage_A_line)
            A.append(heritage_A_odds)
            bovada_A = bovada_A.replace(u'\xa0',' ').replace(u'\xbd','.5')
            bovada_A_line = bovada_A[:bovada_A.find(' ')]
            bovada_A_odds = bovada_A[bovada_A.find(' ') + 1:]
            A.append(bovada_A_line)
            A.append(bovada_A_odds)
            betonline_A = betonline_A.replace(u'\xa0',' ').replace(u'\xbd','.5')
            betonline_A_line = betonline_A[:betonline_A.find(' ')]
            betonline_A_odds = betonline_A[betonline_A.find(' ') + 1:]
            A.append(betonline_A_line)
            A.append(betonline_A_odds)
        else:
            A.append(pinnacle_A.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            A.append(fivedimes_A.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            A.append(heritage_A.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            A.append(bovada_A.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            A.append(betonline_A.replace(u'\xa0',' ').replace(u'\xbd','.5'))
        #H.append(str(date) + '_' + team_A.replace(u'\xa0',' ') + '_' + team_H.replace(u'\xa0',' '))
        H.append(date)
        H.append(time)
        H.append('home')
        H.append(team_H)
        # H.append(pitcher_H)
        # H.append(hand_H)
        H.append(team_A)
        # H.append(pitcher_A)
        # H.append(hand_A)
        if not_ML:
            pinnacle_H = pinnacle_H.replace(u'\xa0',' ').replace(u'\xbd','.5')
            pinnacle_H_line = pinnacle_H[:pinnacle_H.find(' ')]
            pinnacle_H_odds = pinnacle_H[pinnacle_H.find(' ') + 1:]
            H.append(pinnacle_H_line)
            H.append(pinnacle_H_odds)
            fivedimes_H = fivedimes_H.replace(u'\xa0',' ').replace(u'\xbd','.5')
            fivedimes_H_line = fivedimes_H[:fivedimes_H.find(' ')]
            fivedimes_H_odds = fivedimes_H[fivedimes_H.find(' ') + 1:]
            H.append(fivedimes_H_line)
            H.append(fivedimes_H_odds)
            heritage_H = heritage_H.replace(u'\xa0',' ').replace(u'\xbd','.5')
            heritage_H_line = heritage_H[:heritage_H.find(' ')]
            heritage_H_odds = heritage_H[heritage_H.find(' ') + 1:]
            H.append(heritage_H_line)
            H.append(heritage_H_odds)
            bovada_H = bovada_H.replace(u'\xa0',' ').replace(u'\xbd','.5')
            bovada_H_line = bovada_H[:bovada_H.find(' ')]
            bovada_H_odds = bovada_H[bovada_H.find(' ') + 1:]
            H.append(bovada_H_line)
            H.append(bovada_H_odds)
            betonline_H = betonline_H.replace(u'\xa0',' ').replace(u'\xbd','.5')
            betonline_H_line = betonline_H[:betonline_H.find(' ')]
            betonline_H_odds = betonline_H[betonline_H.find(' ') + 1:]
            H.append(betonline_H_line)
            H.append(betonline_H_odds)
        else:
            H.append(pinnacle_H.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            H.append(fivedimes_H.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            H.append(heritage_H.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            H.append(bovada_H.replace(u'\xa0',' ').replace(u'\xbd','.5'))
            H.append(betonline_H.replace(u'\xa0',' ').replace(u'\xbd','.5'))
        
	##For testing purposes..
	#for j in range(len(A)):
		#print 'Test: ', A[j]

        ## Take data from A and H (lists) and put them into DataFrame
        df.loc[counter]   = ([A[j] for j in range(len(A))])
        df.loc[counter+1] = ([H[j] for j in range(len(H))])
        counter += 2
    return df

def select_and_rename(df, text):
    ## Select only useful column names from a DataFrame
    ## Rename column names so that when merged, each df will be unique 
    if text[-2:] == 'ml':
        df = df[['key','time','team','opp_team',
                 'pinnacle','5dimes','heritage','bovada','betonline']]
    ## Change column names to make them unique
        df.columns = ['key',text+'_time','team','opp_team',
                      text+'_PIN',text+'_FD',text+'_HER',text+'_BVD',text+'_BOL']
    else:
        df = df[['key','time','team','opp_team',
                 'pinnacle_line','pinnacle_odds',
                 '5dimes_line','5dimes_odds',
                 'heritage_line','heritage_odds',
                 'bovada_line','bovada_odds',
                 'betonline_line','betonline_odds']]
        df.columns = ['key',text+'_time','team','opp_team',
                      text+'_PIN_line',text+'_PIN_odds',
                      text+'_FD_line',text+'_FD_odds',
                      text+'_HER_line',text+'_HER_odds',
                      text+'_BVD_line',text+'_BVD_odds',
                      text+'_BOL_line',text+'_BOL_odds']
    return df
    

def main(todays_date = None):
    # connectTor()

    ## Get today's lines
    if todays_date is None:
        todays_date = str(date.today()).replace('-','')
    ## change todays_date to be whatever date you want to pull in the format 'yyyymmdd'
    ## One could force user input and if results in blank, revert to today's date. 
    # todays_date = '20140611'

    ## store BeautifulSoup info for parsing
    soup_ml, time_ml = soup_url('ML', todays_date)
    print("getting today's MoneyLine (1/6)")
    soup_rl, time_rl = soup_url('Spreads', todays_date)
    print("getting today's Spreads (2/6)")
    soup_tot, time_tot = soup_url('Totals', todays_date)
    print("getting today's totals (3/6)")
    # soup_1h_ml, time_1h_ml = soup_url('1H', todays_date)
    # print "getting today's 1st-half MoneyLine (4/6)"
    # soup_1h_rl, time_1h_rl = soup_url('1HRL', todays_date)
    # print "getting today's 1st-half RunLine (5/6)"
    # soup_1h_tot, time_1h_tot = soup_url('1Htotal', todays_date)
    # print "getting today's 1st-half totals (6/6)"

    
    #### Each df_xx creates a data frame for a bet type
    print("writing today's MoneyLine (1/6)")
    df_ml = parse_and_write_data(soup_ml, todays_date, time_ml, not_ML = False)
    # print(df_ml)
    ## Change column names to make them unique
    df_ml.columns = ['key','date','ml_time','team',
                     'opp_team',
                     'ml_PIN','ml_FD','ml_HER','ml_BVD','ml_BOL']    

    print("writing today's RunLine (2/6)")
    df_rl = parse_and_write_data(soup_rl, todays_date, time_rl)
    df_rl = select_and_rename(df_rl, 'rl')
    
    print("writing today's totals (3/6)")
    df_tot = parse_and_write_data(soup_tot, todays_date, time_tot)
    df_tot = select_and_rename(df_tot, 'tot')
    
    # print "writing today's 1st-half MoneyLine (4/6)"
    # df_1h_ml = parse_and_write_data(soup_1h_ml, todays_date, time_1h_ml, not_ML = False)
    # df_1h_ml = select_and_rename(df_1h_ml,'1h_ml')
    
    # print "writing today's 1st-half RunLine (5/6)"
    # df_1h_rl = parse_and_write_data(soup_1h_rl, todays_date, time_1h_rl)
    # df_1h_rl = select_and_rename(df_1h_rl,'1h_rl')
    
    # print "writing today's 1st-half totals (6/6)"
    # df_1h_tot = parse_and_write_data(soup_1h_tot, todays_date, time_1h_tot)
    # df_1h_tot = select_and_rename(df_1h_tot,'1h_tot')
    ## Merge all DataFrames together to allow for simple printout
    write_df = df_ml
    write_df = write_df.merge(
                df_rl, how='left', on = ['key','team','opp_team'])
    write_df = write_df.merge(
                df_tot, how='left', on = ['key','team','opp_team'])
    # write_df = write_df.merge(
                # df_1h_ml, how='left', on = ['key','team','pitcher','hand','opp_team'])
    # write_df = write_df.merge(
                # df_1h_rl, how='left', on = ['key','team','pitcher','hand','opp_team'])
    # write_df = write_df.merge(
                # df_1h_tot, how='left', on = ['key','team','pitcher','hand','opp_team'])
    
#     with open(os.getcwd()+'\SBR_NBA_Lines.csv', 'a') as f:
#         write_df.to_csv(f, index=False)#, header = False)
    return(write_df)
  
    ## Code to pull tomorrow's data --- work in progress
    # if time.ml[:2] >= 12:
        # tomorrows_date = str(datetime.date.today() + datetime.timedelta(days=1)).replace('-','')
        # ## store BeautifulSoup info for parsing
        # soup_ml, time_ml = soup_url('ML')
        # print "getting tomorrow's MoneyLine"
        # soup_rl, time_rl = soup_url('RL')
        # print "getting tomorrow's RunLine"
        # soup_tot, time_tot = soup_url('total')
        # print "getting tomorrow's totals"
        # soup_1h_ml, time_1h_ml = soup_url('1H')
        # print "getting tomorrow's 1st-half MoneyLine"
        # soup_1h_rl, time_1h_rl = soup_url('1HRL')
        # print "getting tomorrow's 1st-half RunLine"
        # soup_1h_tot, time_1h_tot = soup_url('1Htotal')
        # print "getting tomorrow's 1st-half totals"

        # parse_and_write_data(soup_ml, todays_date, time_ml, f)
        # parse_and_write_data(soup_rl, todays_date, time_rl, f)
        # parse_and_write_data(soup_tot, todays_date, time_tot, f)
        # parse_and_write_data(soup_1h_ml, todays_date, time_1h_ml, f)
        # parse_and_write_data(soup_1h_rl, todays_date, time_1h_rl, f)
        # parse_and_write_data(soup_1h_tot, todays_date, time_1h_tot, f)

In [3]:
# if __name__ == '__main__':
nbaData = main('20200107')

getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/6
2/6
3/6
4/6
5/6
6/6
writing today's RunLine (2/6)
1/6
2/6
3/6
4/6
5/6
6/6
writing today's totals (3/6)
1/6
2/6
3/6
4/6
5/6
6/6


In [4]:
nbaData[['key', 'date', 'ml_time', 'team', 'opp_team', 'ml_PIN', 'ml_FD',
       'ml_HER', 'ml_BVD', 'ml_BOL', 'rl_time', 'rl_PIN_line', 'rl_PIN_odds',
       'rl_FD_line', 'rl_FD_odds']]

key      date ml_time           team       opp_team ml_PIN  ml_FD  \
0   20200107  11:05:30    away       Portland        Toronto   +123   +125   
1   20200107  11:05:30    home        Toronto       Portland   -136   -135   
2   20200107  11:05:30    away        Detroit      Cleveland   -164   -155   
3   20200107  11:05:30    home      Cleveland        Detroit   +147   +145   
4   20200107  11:05:30    away  Oklahoma City       Brooklyn   -103   +102   
5   20200107  11:05:30    home       Brooklyn  Oklahoma City   -107   -112   
6   20200107  11:05:30    away      Minnesota        Memphis   +153   +167   
7   20200107  11:05:30    home        Memphis      Minnesota   -171   -178   
8   20200107  11:05:30    away     Sacramento        Phoenix   +228   +245   
9   20200107  11:05:30    home        Phoenix     Sacramento   -262   -265   
10  20200107  11:05:30    away       New York    L.A. Lakers   +840   +900   
11  20200107  11:05:30    home    L.A. Lakers       New York  -1225  -1100   

   ml_HER ml_BVD ml_BOL rl_time rl_PIN_line rl_PIN_odds rl_FD_line rl_FD_odds  
0    +125   +120   +117    away        +2.5        -103       +2.5       -108  
1    -145   -140   -137    home        -2.5        -107       -2.5       -102  
2    -170   -165   -170    away        -3.5        -105       -3.5       -105  
3    +150   +145   +150    home        +3.5        -105       +3.5       -105  
4    -110   -110   -114    away          +1        -108         +1       -103  
5    -110   -110   -106    home          -1        -102         -1       -107  
6    +160   +160   +160    away        +4.5        -113       +4.5       -108  
7    -185   -180   -180    home        -4.5        +102       -4.5       -102  
8    +235   +205   +225    away        +6.5        -105       +6.5       -105  
9    -285   -245   -275    home        -6.5        -105       -6.5       -105  
10   +825   +725   +825    away         +13        -105      +13.5       -108  
11  -1175  -1300  -1400    home         -13        -105      -13.5       -102

In [5]:
str(date.today()).replace('-','')

'20200109'

In [6]:
l=11
print(nbaData['ml_PIN'][l])
plusMinus = nbaData['ml_PIN'][l][0]
if plusMinus is '+':
    odds = int(nbaData['ml_PIN'][l][1:]) / 100 + 1
elif plusMinus is '-':
    odds = 100 / int(nbaData['ml_PIN'][l][1:]) + 1
else:
    print('error')
    odds = 0.5
print(odds)
print(1/odds)


-1225
1.0816326530612246
0.9245283018867924


In [7]:
nbaData['ml_PIN_sign'] = [s[0] for s in nbaData['ml_PIN']]

In [8]:
nbaData['ml_PIN_dec'] = 0
nbaData['ml_PIN_prob'] = 0

nbaData['ml_PIN_int'] = [int(s[1:]) for s in nbaData['ml_PIN']]

nbaData.loc[nbaData['ml_PIN_sign'] == '+', 'ml_PIN_dec'] = \
    nbaData.loc[nbaData['ml_PIN_sign'] == '+', 'ml_PIN_int']/100 + 1

nbaData.loc[nbaData['ml_PIN_sign'] == '-', 'ml_PIN_dec'] = \
    100 / nbaData.loc[nbaData['ml_PIN_sign'] == '-', 'ml_PIN_int'] + 1

nbaData['ml_PIN_prob'] = 1/ nbaData['ml_PIN_dec']
# nbaData.loc[nbaData['ml_PIN_sign'] == '+', 'ml_PIN_int'] = \
#         [int(s[1:]) for s in nbaData.loc[nbaData['ml_PIN_sign'] == '+', 'ml_PIN']]



In [9]:
nbaData.loc[nbaData.ml_PIN_prob > 0.6, ['ml_time', 'team', 'ml_PIN_dec', 'ml_PIN_prob']]

ml_time         team  ml_PIN_dec  ml_PIN_prob
2     away      Detroit    1.609756     0.621212
7     home      Memphis    1.584795     0.630996
9     home      Phoenix    1.381679     0.723757
11    home  L.A. Lakers    1.081633     0.924528

In [10]:
(1.609756 + 1.584795 + 1.081633) - 4

0.27618399999999976

In [11]:
nbaData = main('20200108')

getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9
writing today's RunLine (2/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9
writing today's totals (3/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9


In [12]:
nbaData[['key', 'date', 'ml_time', 'team', 'opp_team', 'ml_PIN', 'ml_FD',
       'ml_HER', 'ml_BVD', 'ml_BOL']]

key      date ml_time          team      opp_team ml_PIN  ml_FD  \
0   20200108  11:05:41    away    Washington       Orlando   +409   +400   
1   20200108  11:05:41    home       Orlando    Washington   -508   -450   
2   20200108  11:05:41    away         Miami       Indiana   +106   +110   
3   20200108  11:05:41    home       Indiana         Miami   -117   -120   
4   20200108  11:05:41    away       Toronto     Charlotte   -136   -140   
5   20200108  11:05:41    home     Charlotte       Toronto   +123   +130   
6   20200108  11:05:41    away   San Antonio        Boston   +269   +265   
7   20200108  11:05:41    home        Boston   San Antonio   -315   -295   
8   20200108  11:05:41    away       Houston       Atlanta   -320   -315   
9   20200108  11:05:41    home       Atlanta       Houston   +273   +285   
10  20200108  11:05:41    away        Denver        Dallas   +139   +145   
11  20200108  11:05:41    home        Dallas        Denver   -154   -155   
12  20200108  11:05:41    away       Chicago   New Orleans   +142   +145   
13  20200108  11:05:41    home   New Orleans       Chicago   -158   -155   
14  20200108  11:05:41    away      New York          Utah   +853   +900   
15  20200108  11:05:41    home          Utah      New York  -1250  -1100   
16  20200108  11:05:41    away     Milwaukee  Golden State  -1052  -1020   
17  20200108  11:05:41    home  Golden State     Milwaukee   +745   +820   

   ml_HER ml_BVD ml_BOL  
0    +445   +375   +380  
1    -570   -550   -500  
2    -105   -125   +100  
3    -115   +105   -120  
4    -150   -145   -142  
5    +130   +125   +122  
6    +275   +275   +265  
7    -345   -350   -335  
8    -365   -330   -350  
9    +295   +265   +275  
10   +140   +125   +140  
11   -160   -145   -160  
12   +140   +135   +140  
13   -160   -155   -160  
14   +850   +775   +800  
15  -1200  -1500  -1350  
16  -1080  -1400  -1350  
17   +780   +750   +800

In [13]:
def calc_odds_prob(df, broker = 'PIN'):
    
    
    broker = 'ml_' + broker
    broker_sign = broker + '_sign'
    broker_dec = broker + '_dec'
    broker_int = broker + '_int'
    broker_prob = broker + '_prob'
    
    df[broker_sign] = [s[0] for s in df[broker]]

    df[broker + '_dec'] = 0

    df[broker + '_int'] = [int(s[1:]) for s in df[broker]]

    df.loc[df[broker_sign] == '+', broker_dec] = \
        df.loc[df[broker_sign] == '+', broker_int]/100 + 1

    df.loc[df[broker_sign] == '-', broker_dec] = \
        100 / df.loc[df[broker_sign] == '-', broker_int] + 1

    df[broker_prob] = 1/ df[broker_dec]
    
    return(df)

def calc_agg_odds_prob(df, brokers = ['PIN', 'HER', 'FD', 'BOL', 'BVD']):
    
    # calculate odds and prob
    brokers_with_odds = []
    for b in brokers:
        try:
            print('calculating odds for broker {}'.format(b))
            df = calc_odds_prob(df, b)
            brokers_with_odds.append(b)
        except:
            print('odds missing for broker {}, skipping'.format(b))
    
    # agg odds and prob
    df['ml_AGG_dec'] = df[['ml_' + b + '_dec' for b in brokers_with_odds]].sum(axis = 1) / len(brokers_with_odds)
    df['ml_MAX_dec'] = df[['ml_' + b + '_dec' for b in brokers_with_odds]].max(axis = 1)
    
    df['ml_AGG_prob'] = df[['ml_' + b + '_prob' for b in brokers_with_odds]].sum(axis = 1) / len(brokers_with_odds)
    df['ml_MAX_prob'] = df[['ml_' + b + '_prob' for b in brokers_with_odds]].max(axis = 1)
    
    return(df, brokers_with_odds)

In [14]:
nbaData = calc_odds_prob(nbaData)

In [15]:
nbaData[['key', 'date', 'ml_time', 'team', 'opp_team', 'ml_PIN',
         'ml_PIN_sign', 'ml_PIN_dec', 'ml_PIN_int', 'ml_PIN_prob']]

key      date ml_time          team      opp_team ml_PIN ml_PIN_sign  \
0   20200108  11:05:41    away    Washington       Orlando   +409           +   
1   20200108  11:05:41    home       Orlando    Washington   -508           -   
2   20200108  11:05:41    away         Miami       Indiana   +106           +   
3   20200108  11:05:41    home       Indiana         Miami   -117           -   
4   20200108  11:05:41    away       Toronto     Charlotte   -136           -   
5   20200108  11:05:41    home     Charlotte       Toronto   +123           +   
6   20200108  11:05:41    away   San Antonio        Boston   +269           +   
7   20200108  11:05:41    home        Boston   San Antonio   -315           -   
8   20200108  11:05:41    away       Houston       Atlanta   -320           -   
9   20200108  11:05:41    home       Atlanta       Houston   +273           +   
10  20200108  11:05:41    away        Denver        Dallas   +139           +   
11  20200108  11:05:41    home        Dallas        Denver   -154           -   
12  20200108  11:05:41    away       Chicago   New Orleans   +142           +   
13  20200108  11:05:41    home   New Orleans       Chicago   -158           -   
14  20200108  11:05:41    away      New York          Utah   +853           +   
15  20200108  11:05:41    home          Utah      New York  -1250           -   
16  20200108  11:05:41    away     Milwaukee  Golden State  -1052           -   
17  20200108  11:05:41    home  Golden State     Milwaukee   +745           +   

    ml_PIN_dec  ml_PIN_int  ml_PIN_prob  
0     5.090000         409     0.196464  
1     1.196850         508     0.835526  
2     2.060000         106     0.485437  
3     1.854701         117     0.539171  
4     1.735294         136     0.576271  
5     2.230000         123     0.448430  
6     3.690000         269     0.271003  
7     1.317460         315     0.759036  
8     1.312500         320     0.761905  
9     3.730000         273     0.268097  
10    2.390000         139     0.418410  
11    1.649351         154     0.606299  
12    2.420000         142     0.413223  
13    1.632911         158     0.612403  
14    9.530000         853     0.104932  
15    1.080000        1250     0.925926  
16    1.095057        1052     0.913194  
17    8.450000         745     0.118343

In [16]:
nbaData.loc[nbaData.ml_PIN_prob > 0.6, ['ml_time', 'team', 'ml_PIN_dec', 'ml_PIN_prob']]

ml_time         team  ml_PIN_dec  ml_PIN_prob
1     home      Orlando    1.196850     0.835526
7     home       Boston    1.317460     0.759036
8     away      Houston    1.312500     0.761905
11    home       Dallas    1.649351     0.606299
13    home  New Orleans    1.632911     0.612403
15    home         Utah    1.080000     0.925926
16    away    Milwaukee    1.095057     0.913194

In [17]:
nbaData, brokers_with_odds = calc_agg_odds_prob(nbaData)

calculating odds for broker PIN
calculating odds for broker HER
calculating odds for broker FD
calculating odds for broker BOL
calculating odds for broker BVD


In [18]:
cols = ['ml_time', 'team', 'ml_AGG_dec', 'ml_AGG_prob'] + ['ml_' + b + '_prob' for b in brokers_with_odds]

In [19]:
nbaData[cols]

ml_time          team  ml_AGG_dec  ml_AGG_prob  ml_PIN_prob  ml_HER_prob  \
0     away    Washington    5.018000     0.199762     0.196464     0.183486   
1     home       Orlando    1.195266     0.836788     0.835526     0.850746   
2     away         Miami    1.982476     0.505876     0.485437     0.512195   
3     home       Indiana    1.888187     0.530554     0.539171     0.534884   
4     away       Toronto    1.702025     0.587644     0.576271     0.600000   
5     home     Charlotte    2.260000     0.442578     0.448430     0.434783   
6     away   San Antonio    3.698000     0.270456     0.271003     0.266667   
7     home        Boston    1.306104     0.765809     0.759036     0.775281   
8     away       Houston    1.298536     0.770221     0.761905     0.784946   
9     home       Atlanta    3.786000     0.264328     0.268097     0.253165   
10    away        Denver    2.378000     0.420870     0.418410     0.416667   
11    home        Dallas    1.646833     0.607350     0.606299     0.615385   
12    away       Chicago    2.404000     0.416050     0.413223     0.416667   
13    home   New Orleans    1.634647     0.611772     0.612403     0.615385   
14    away      New York    9.356000     0.107118     0.104932     0.105263   
15    home          Utah    1.078997     0.926841     0.925926     0.923077   
16    away     Milwaukee    1.086238     0.920706     0.913194     0.915254   
17    home  Golden State    8.790000     0.113887     0.118343     0.113636   

    ml_FD_prob  ml_BOL_prob  ml_BVD_prob  
0     0.200000     0.208333     0.210526  
1     0.818182     0.833333     0.846154  
2     0.476190     0.500000     0.555556  
3     0.545455     0.545455     0.487805  
4     0.583333     0.586777     0.591837  
5     0.434783     0.450450     0.444444  
6     0.273973     0.273973     0.266667  
7     0.746835     0.770115     0.777778  
8     0.759036     0.777778     0.767442  
9     0.259740     0.266667     0.273973  
10    0.408163     0.416667     0.444444  
11    0.607843     0.615385     0.591837  
12    0.408163     0.416667     0.425532  
13    0.607843     0.615385     0.607843  
14    0.100000     0.111111     0.114286  
15    0.916667     0.931034     0.937500  
16    0.910714     0.931034     0.933333  
17    0.108696     0.111111     0.117647

In [20]:
nbaData.loc[nbaData.ml_AGG_prob > 0.6, cols]

ml_time         team  ml_AGG_dec  ml_AGG_prob  ml_PIN_prob  ml_HER_prob  \
1     home      Orlando    1.195266     0.836788     0.835526     0.850746   
7     home       Boston    1.306104     0.765809     0.759036     0.775281   
8     away      Houston    1.298536     0.770221     0.761905     0.784946   
11    home       Dallas    1.646833     0.607350     0.606299     0.615385   
13    home  New Orleans    1.634647     0.611772     0.612403     0.615385   
15    home         Utah    1.078997     0.926841     0.925926     0.923077   
16    away    Milwaukee    1.086238     0.920706     0.913194     0.915254   

    ml_FD_prob  ml_BOL_prob  ml_BVD_prob  
1     0.818182     0.833333     0.846154  
7     0.746835     0.770115     0.777778  
8     0.759036     0.777778     0.767442  
11    0.607843     0.615385     0.591837  
13    0.607843     0.615385     0.607843  
15    0.916667     0.931034     0.937500  
16    0.910714     0.931034     0.933333

In [21]:
# read in historical results

In [84]:
# transform to odds data format

TeamNameDict = {}

TeamNameDict['NewOrleans'] = 'New Orleans'
TeamNameDict['Toronto'] ='Toronto'
TeamNameDict['LALakers'] = 'L.A. Lakers'
TeamNameDict['LAClippers'] = 'L.A. Clippers'
TeamNameDict['Detroit'] = 'Detroit' 
TeamNameDict['Indiana'] = 'Indiana'
TeamNameDict['Cleveland'] = 'Cleveland'
TeamNameDict['Orlando'] = 'Orlando'
TeamNameDict['Chicago'] = 'Chicago'
TeamNameDict['Charlotte'] = 'Charlotte'
TeamNameDict['Boston'] = 'Boston'
TeamNameDict['Philadelphia'] = 'Philadelphia'
TeamNameDict['Memphis'] = 'Memphis'
TeamNameDict['Miami'] = 'Miami'
TeamNameDict['Minnesota'] = 'Minnesota'
TeamNameDict['Brooklyn'] = 'Brooklyn'
TeamNameDict['NewYork'] = 'NewYork'
TeamNameDict['SanAntonio'] = 'SanAntonio'
TeamNameDict['Washington'] = 'Washington'
TeamNameDict['Dallas'] = 'Dallas'
TeamNameDict['OklahomaCity'] = 'Oklahoma City'
TeamNameDict['Utah'] = 'Utah'
TeamNameDict['Sacramento'] = 'Sacramento'
TeamNameDict['Phoenix'] = 'Phoenix'
TeamNameDict['Denver'] = 'Denver'
TeamNameDict['Portland'] = 'Portland'
TeamNameDict['Atlanta'] = 'Atlanta'
TeamNameDict['Milwaukee'] = 'Milwaukee'
TeamNameDict['Houston'] = 'Houston'
TeamNameDict['GoldenState'] = 'GoldenState'

def transform_results_format(df):
    
    df['HomeAway'] = 'home'
    df.loc[df.VH == 'V', 'HomeAway'] = 'away'

    df = pd.merge(left = df,
             left_on = 'Team',
             right = pd.DataFrame.from_dict(TeamNameDict , orient='index', columns=['TeamName']),
             right_index = True,
             how = 'left')

    df['date'] = ['2019' + str(d) for d in df.Date]


    # who won the game

    df['WinLoss'] = 0
    WinLossColId = df.columns.get_loc('WinLoss')
    for i in range(df.shape[0]):
        if i % 2 == 0:
            # check row below
            r = 1
        if i % 2 == 1:
            r = -1
            
        df.iloc[i, WinLossColId] = df.iloc[i]['Final'] > df.iloc[i+r]['Final']

    return(df)
#     #         check row above
#             nbaResults.iloc[i, WinLossColId] = nbaResults.iloc[i]['Final'] > nbaResults.iloc[i-1]['Final']


In [91]:
nbaResults = pd.read_csv('data/nba_odds_2019_20.csv')

In [92]:
nbaResults = transform_results_format(nbaResults)
nbaResults.head()

Date  Rot VH        Team  1st  2nd  3rd  4th  Final   Open  Close   ML  \
0  1022  501  V  NewOrleans   30   31   25   31    122  231.5  229.5  230   
1  1022  502  H     Toronto   27   29   32   29    130    6.5    6.5 -280   
2  1022  503  V    LALakers   25   29   31   17    102    227    3.5 -180   
3  1022  504  H  LAClippers   22   40   23   27    112    1.5    224  150   
4  1023  505  V     Detroit   27   27   29   36    119    210    211  240   

      2H HomeAway       TeamName      date WinLoss  
0    113     away    New Orleans  20191022   False  
1      6     home        Toronto  20191022    True  
2      5     away    L.A. Lakers  20191022   False  
3  110.5     home  L.A. Clippers  20191022    True  
4    104     away        Detroit  20191023    True

In [93]:
nbaData.head()

key      date ml_time        team    opp_team ml_PIN ml_FD ml_HER  \
0  20200108  11:05:41    away  Washington     Orlando   +409  +400   +445   
1  20200108  11:05:41    home     Orlando  Washington   -508  -450   -570   
2  20200108  11:05:41    away       Miami     Indiana   +106  +110   -105   
3  20200108  11:05:41    home     Indiana       Miami   -117  -120   -115   
4  20200108  11:05:41    away     Toronto   Charlotte   -136  -140   -150   

  ml_BVD ml_BOL  ... ml_BOL_int ml_BOL_prob ml_BVD_sign ml_BVD_dec ml_BVD_int  \
0   +375   +380  ...        380    0.208333           +   4.750000        375   
1   -550   -500  ...        500    0.833333           -   1.181818        550   
2   -125   +100  ...        100    0.500000           -   1.800000        125   
3   +105   -120  ...        120    0.545455           +   2.050000        105   
4   -145   -142  ...        142    0.586777           -   1.689655        145   

  ml_BVD_prob ml_AGG_dec ml_MAX_dec ml_AGG_prob ml_MAX_prob  
0    0.210526   5.018000   5.450000    0.199762    0.210526  
1    0.846154   1.195266   1.222222    0.836788    0.850746  
2    0.555556   1.982476   2.100000    0.505876    0.555556  
3    0.487805   1.888187   2.050000    0.530554    0.545455  
4    0.591837   1.702025   1.735294    0.587644    0.600000  

[5 rows x 56 columns]

In [88]:
d = nbaResults.date.unique()[0]

In [104]:
# for d in nbaResults.date.unique():
    
def get_odds_dates(dates):
    
    df_list = []
    
    for d in dates:
        # get the odds
        df_list.append(main(d))
        
    out = pd.concat(df_list)
    
    return(out)

In [105]:
all_odds = get_odds_dates(nbaResults.date.unique())

getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/2
2/2
writing today's RunLine (2/6)
1/2
2/2
writing today's totals (3/6)
1/2
2/2
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
writing today's RunLine (2/6)
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
writing today's totals (3/6)
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/3
2/3
3/3
writing today's RunLine (2/6)
1/3
2/3
3/3
writing today's totals (3/6)
1/3
2/3
3/3
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9
writing today's RunLine (2/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/

5/11
6/11
7/11
8/11
9/11
10/11
11/11
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/2
2/2
writing today's RunLine (2/6)
1/2
2/2
writing today's totals (3/6)
1/2
2/2
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
writing today's RunLine (2/6)
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
writing today's totals (3/6)
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
writing today's RunLine (2/6)
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
writing today's totals (3/6)
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3

3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9
writing today's RunLine (2/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9
writing today's totals (3/6)
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/5
2/5
3/5
4/5
5/5
writing today's RunLine (2/6)
1/5
2/5
3/5
4/5
5/5
writing today's totals (3/6)
1/5
2/5
3/5
4/5
5/5
getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
writing today's RunLine (2/6)
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
writing today's totals (3/6)
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11


In [107]:
all_odds

key      date ml_time           team       opp_team ml_PIN ml_FD  \
0   20191022  12:12:47    away    New Orleans        Toronto   +237  +245   
1   20191022  12:12:47    home        Toronto    New Orleans   -273  -265   
2   20191022  12:12:47    away    L.A. Lakers  L.A. Clippers   -165  -160   
3   20191022  12:12:47    home  L.A. Clippers    L.A. Lakers   +148  +150   
0   20191023  12:12:57    away        Detroit        Indiana   +261  +280   
..       ...       ...     ...            ...            ...    ...   ...   
17  20191223  12:25:49    home       Portland    New Orleans   -234  -230   
18  20191223  12:25:49    away        Houston     Sacramento   -251  -245   
19  20191223  12:25:49    home     Sacramento        Houston   +219  +225   
20  20191223  12:25:49    away      Minnesota   Golden State   -102  -105   
21  20191223  12:25:49    home   Golden State      Minnesota   -108  -105   

   ml_HER ml_BVD ml_BOL  ... tot_PIN_line tot_PIN_odds tot_FD_line  \
0    +245   +230   +235  ...        229.5         -109       229.5   
1    -305   -280   -280  ...        229.5         -101       229.5   
2    -180   -180   -168  ...          224         -101         224   
3    +155   +160   +148  ...          224         -109         224   
0    +255   +260   +250  ...        211.5         -101       211.5   
..    ...    ...    ...  ...          ...          ...         ...   
17   -265   -250   -240  ...          230         -103       230.5   
18   -275   -275   -260  ...          229         +101         230   
19   +225   +225   +220  ...          229         -112         230   
20   -110   -105   +100  ...          221         -104         221   
21   -110   -115   -120  ...          221         -106         221   

   tot_FD_odds tot_HER_line tot_HER_odds tot_BVD_line tot_BVD_odds  \
0         -103          229         -105        228.5         -115   
1         -107          229         -105        228.5         -105   
2         -105          224         -105          225         -105   
3         -105          224         -105          225         -115   
0         -102          211         -105          211         -110   
..         ...          ...          ...          ...          ...   
17        -105          230         -105        230.5         -110   
18        -105        230.5         -105        230.5         -105   
19        -105        230.5         -105        230.5         -115   
20        -105          221         -105          222         -105   
21        -105          221         -105          222         -115   

   tot_BOL_line tot_BOL_odds  
0         229.5         -110  
1         229.5         -110  
2         223.5         -112  
3         223.5         -108  
0           211         -111  
..          ...          ...  
17          230         -105  
18          230         -105  
19          230         -115  
20          221         -110  
21          221         -110  

[908 rows x 32 columns]

In [108]:
def merge_odds_results(AllOddsDf, nbaResultsDf):

# convert odds
    AllOddsDf, brokers_with_odds = calc_agg_odds_prob(AllOddsDf)

    # join
    join_cols_left = ['HomeAway', 'TeamName', 'date'] # results
    join_cols_right = ['ml_time', 'team', 'key'] # odds

    nbaResultsDf = pd.merge(left = nbaResultsDf,
            left_on = join_cols_left,
            right = AllOddsDf,
            right_on = join_cols_right,
            how = 'left')
    
    return(nbaResultsDf)

In [109]:
nbaResultsOdds = merge_odds_results(all_odds, nbaResults)

calculating odds for broker PIN
calculating odds for broker HER
calculating odds for broker FD
calculating odds for broker BOL
calculating odds for broker BVD


In [207]:
nbaResultsOdds.head()

Date  Rot VH        Team  1st  2nd  3rd  4th  Final   Open  ... ml_BOL_int  \
0  1022  501  V  NewOrleans   30   31   25   31    122  231.5  ...      235.0   
1  1022  502  H     Toronto   27   29   32   29    130    6.5  ...      280.0   
2  1022  503  V    LALakers   25   29   31   17    102    227  ...      168.0   
3  1022  504  H  LAClippers   22   40   23   27    112    1.5  ...      148.0   
4  1023  505  V     Detroit   27   27   29   36    119    210  ...      250.0   

   ml_BOL_prob ml_BVD_sign ml_BVD_dec ml_BVD_int ml_BVD_prob ml_AGG_dec  \
0     0.298507           +   3.300000      230.0    0.303030   3.384000   
1     0.736842           -   1.357143      280.0    0.736842   1.357163   
2     0.626866           -   1.555556      180.0    0.642857   1.587482   
3     0.403226           +   2.600000      160.0    0.384615   2.522000   
4     0.285714           +   3.600000      260.0    0.277778   3.612000   

  ml_MAX_dec ml_AGG_prob ml_MAX_prob  
0   3.450000    0.295597    0.303030  
1   1.377358    0.736940    0.753086  
2   1.625000    0.630121    0.642857  
3   2.600000    0.396645    0.403226  
4   3.800000    0.277070    0.285714  

[5 rows x 73 columns]

In [219]:
cols = ['key', 'ml_time', 'team', 'ml_AGG_dec', 'ml_AGG_prob'] + ['WinLoss']
nbaResultsOdds[cols]

key ml_time           team  ml_AGG_dec  ml_AGG_prob WinLoss
0    20191022    away    New Orleans    3.384000     0.295597   False
1    20191022    home        Toronto    1.357163     0.736940    True
2    20191022    away    L.A. Lakers    1.587482     0.630121   False
3    20191022    home  L.A. Clippers    2.522000     0.396645    True
4    20191023    away        Detroit    3.612000     0.277070    True
..        ...     ...            ...         ...          ...     ...
903  20191223    home     Sacramento    3.228000     0.309811   False
904  20191223    away    New Orleans    3.082000     0.324550    True
905  20191223    home       Portland    1.411232     0.708753   False
906  20191223    away      Minnesota    1.958849     0.510630   False
907       NaN     NaN            NaN         NaN          NaN    True

[908 rows x 6 columns]

In [209]:
betsPlaced = nbaResultsOdds.loc[nbaResultsOdds.ml_AGG_prob > 0.6, cols]

In [210]:
betsPlaced = nbaResultsOdds.loc[nbaResultsOdds.ml_AGG_prob > 0.65, cols]
(betsPlaced.loc[betsPlaced.WinLoss].ml_AGG_dec.sum() - betsPlaced.shape[0]) / betsPlaced.shape[0]

0.0647956725058247

In [260]:
betsPlaced = nbaResultsOdds.loc[nbaResultsOdds.ml_AGG_prob > 0.65, cols]
exp = 22
total_wagered = (betsPlaced.ml_AGG_dec ** exp).sum()
total_return = (betsPlaced.loc[betsPlaced.WinLoss].ml_AGG_dec ** (1+exp)).sum()
precent_return = (total_return -  total_wagered) / total_wagered

betsPlaced['wager'] = betsPlaced['ml_AGG_dec'] ** exp
betsPlaced['return'] = betsPlaced.ml_AGG_dec ** (1+exp) * betsPlaced.WinLoss

betsPlaced['PnL'] = betsPlaced['return'] - betsPlaced['wager']

print('total wagered {}'.format(total_wagered))
print('total return {}'.format(total_return))
print('profit {}'.format(total_return - total_wagered))
print('% return {}'.format(precent_return))
print('max bet ${}'.format((betsPlaced.ml_AGG_dec ** exp).max()))
print('% winning days {}'.format((betsPlaced.groupby('key')['PnL'].sum() > 0).mean()))


print('\nwager summary:')
print(betsPlaced.groupby('key')['wager'].sum().describe())

print('\ndaily return summary:')
print((betsPlaced.groupby('key')['return'].sum() - betsPlaced.groupby('key')['wager'].sum()).describe())



total wagered 456226.33012129355
total return 519678.759766196
profit 63452.42964490247
% return 0.13908103381063702
max bet $12633.339622037116
% winning days 0.7540983606557377

wager summary:
count       61.000000
mean      7479.120166
std       8126.267498
min          7.588401
25%       1840.163585
50%       4099.094922
75%      10084.507604
max      32155.526229
Name: wager, dtype: float64

daily return summary:
count       61.000000
mean      1040.203765
std       4553.492646
min     -10610.488426
25%          0.732246
50%        471.346939
75%       2210.333414
max      15562.033193
dtype: float64


In [241]:
betsPlaced['PnL'] = betsPlaced['return'] - betsPlaced['wager']

In [251]:
(betsPlaced.groupby('key')['PnL'].sum() > 0).mean()

0.7540983606557377